# Tariff Shock Options Strategy (SMH)  
## Notebook 1: Data Pull (Prices + Options Chain)

## 1. Pull SMH price history  
## 2. Select nearest 30-day expiry  
## 3. Download and clean options chain  
## 4. Save cleaned datasets to /data


In [3]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent  # notebooks/ -> project root
sys.path.insert(0, str(PROJECT_ROOT))

print("Project root added to path:", PROJECT_ROOT)


Project root added to path: c:\Users\nicho\tariff-options-strategy


In [4]:
import pandas as pd

from src.tariff_strategy.data.market_data import (
    fetch_price_history,
    list_option_expiries,
    nearest_expiry,
    fetch_options_chain,
    clean_chain,
)


In [5]:
TICKER = "SMH"

prices = fetch_price_history(TICKER, period="2y")
prices.head()


Price,Date,Adj Close,Close,High,Low,Open,Volume
Ticker,,SMH,SMH,SMH,SMH,SMH,SMH
0,2024-01-17,173.330032,174.639999,174.929993,171.490005,174.479996,6751200
1,2024-01-18,178.957520,180.309998,180.550003,177.410004,179.669998,11983800
2,2024-01-19,185.775970,187.179993,187.309998,181.419998,182.389999,12742400
3,2024-01-22,186.639450,188.050003,189.979996,186.470001,188.809998,8308900
4,2024-01-23,187.592255,189.009995,189.169998,186.089996,188.020004,6031200


In [6]:
last = prices["Adj Close"].iloc[-1]
s0 = float(last.iloc[0]) if hasattr(last, "iloc") else float(last)

print(f"S0 (latest Adj Close): {s0:.2f}")


S0 (latest Adj Close): 400.39


In [7]:
TARGET_DAYS = 30

expiries = list_option_expiries(TICKER)
chosen_expiry = nearest_expiry(expiries, target_days=TARGET_DAYS)

print("First 10 expiries:", expiries[:10])
print(f"Chosen expiry near {TARGET_DAYS} days:", chosen_expiry)


First 10 expiries: ['2026-01-23', '2026-01-30', '2026-02-06', '2026-02-13', '2026-02-20', '2026-02-27', '2026-03-20', '2026-04-17', '2026-05-15', '2026-06-18']
Chosen expiry near 30 days: 2026-02-13


In [8]:
chain = fetch_options_chain(TICKER, chosen_expiry)
chain = clean_chain(chain)

calls = chain.calls
puts = chain.puts

print("Calls shape:", calls.shape)
print("Puts shape:", puts.shape)

calls.head()


Calls shape: (42, 11)
Puts shape: (45, 11)


,contractSymbol,strike,lastPrice,bid,ask,mid,spread,volume,openInterest,impliedVolatility,inTheMoney
0,SMH260213C00295000,295.0,84.80,104.30,108.85,106.575,4.55,NaN,1,0.655033,True
1,SMH260213C00305000,305.0,75.40,95.25,98.25,96.750,3.00,NaN,25,0.612186,True
2,SMH260213C00310000,310.0,70.55,89.60,94.15,91.875,4.55,NaN,10,0.593144,True
3,SMH260213C00315000,315.0,65.55,85.50,88.45,86.975,2.95,NaN,27,0.571171,True
4,SMH260213C00325000,325.0,52.82,75.80,78.75,77.275,2.95,42.0,21,0.532231,True


In [9]:
puts.head()


,contractSymbol,strike,lastPrice,bid,ask,mid,spread,volume,openInterest,impliedVolatility,inTheMoney
0,SMH260213P00265000,265.0,0.25,0.00,0.31,0.155,0.31,9.0,9,0.622074,False
1,SMH260213P00270000,270.0,0.17,0.08,0.24,0.160,0.16,1.0,23,0.599613,False
2,SMH260213P00280000,280.0,0.33,0.06,0.41,0.235,0.35,4.0,5,0.577153,False
3,SMH260213P00285000,285.0,0.62,0.10,0.45,0.275,0.35,30.0,15,0.563969,False
4,SMH260213P00290000,290.0,0.32,0.14,0.45,0.295,0.31,4.0,6,0.543950,False


In [10]:
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

prices.to_csv(DATA_DIR / "smh_prices.csv", index=False)
calls.to_csv(DATA_DIR / f"smh_calls_{chosen_expiry}.csv", index=False)
puts.to_csv(DATA_DIR / f"smh_puts_{chosen_expiry}.csv", index=False)

print("Saved:")
print("-", DATA_DIR / "smh_prices.csv")
print("-", DATA_DIR / f"smh_calls_{chosen_expiry}.csv")
print("-", DATA_DIR / f"smh_puts_{chosen_expiry}.csv")


Saved:
- data\smh_prices.csv
- data\smh_calls_2026-02-13.csv
- data\smh_puts_2026-02-13.csv


In [11]:
summary = pd.DataFrame({
    "ticker": [TICKER],
    "S0": [s0],
    "chosen_expiry": [chosen_expiry],
    "n_calls": [len(calls)],
    "n_puts": [len(puts)],
})

summary


,ticker,S0,chosen_expiry,n_calls,n_puts
0,SMH,400.390015,2026-02-13,42,45
